In [313]:
from pathlib import Path
from medkit.core.text import TextDocument
from medkit.text.segmentation import SentenceTokenizer
from medkit.text.ner import RegexpMatcher, RegexpMatcherRule , RegexpMatcherNormalization
from medkit.text.context import NegationDetector, NegationDetectorRule
from medkit.text.segmentation import SyntagmaTokenizer
from medkit.text.context import FamilyDetector
from unidecode import unidecode
import pandas as pd
import os
import re

In [314]:
"""
La fonction preprocess_text effectue le prétraitement d'un texte en remplaçant certains caractères spéciaux 
par leur équivalent en ASCII. Elle convertit le texte donné en entrée en ASCII et normalise les espaces en remplaçant les 
espaces multiples par un seul espace. La fonction renvoie Le texte prétraité.

"""
def preprocess_text(text):
    # Convertir les caractères spéciaux spécifiques avant la conversion en ASCII
    text = re.sub(r'n°', 'numero', text)  # Remplace "n°" par "numero"
    text = re.sub(r'/d°', 'deg', text)  # Remplace "/d°" par "deg"

    # Convertir le texte en ASCII
    ascii_text = unidecode(text)  # Convertit les caractères Unicode en ASCII

    # Normaliser les espaces en remplaçant les espaces multiples par un seul espace
    ascii_text = re.sub(r'\s+', ' ', ascii_text)  # Remplace plusieurs espaces par un seul espace

    return ascii_text  # Retourne le texte prétraité en ASCII

In [315]:
"""
La fonction splitting_doc_sentences prend un document en entrée et utilise un tokenizer de phrases pour segmenter le document
en phrases individuelles, en utilisant des caractères de ponctuation tels que ".", "?", et "!". Les phrases segmentées
sont renvoyées en sortie de la fonction.

"""

def splitting_doc_sentences(doc):
    sent_tokenizer = SentenceTokenizer(
        output_label="sentence",
        punct_chars=[".", "?", "!"],
    )
    sentences = sent_tokenizer.run([doc.raw_segment])
    return sentences

In [316]:
"""
La fonction splitting_doc_syntagmas prend des phrases (sentences) en entrée et utilise un tokenizer 
de syntagmes pour segmenter à nouveau les sentences en utilisant en utilisant les séparateurs tels que
"mais" et "et".. Les syntagmas sont renvoyées en sortie de la fonction.

"""
def splitting_doc_syntagmas(sentences):

    # On sépare les phrases si il y'a "mais" "et"
    synt_tokenizer = SyntagmaTokenizer(
        output_label="sentence",
        separators=[r"\bmais\b", r"\bet\b"],
    )
    syntagmas = synt_tokenizer.run(sentences)
    return syntagmas

In [317]:
"""
La fonction finding_entities prend en entrée une liste de phrases (sentences) et utilise
des règles de correspondance basées sur des expressions régulières pour extraire des
entités spécifiques du texte. Les entités recherchées sont liées au tabac, à l'alcool, 
au statut marital et aux antécédents. Elle renvoie les entités extraites et les règles 
regex qui ont été utilisées.
"""
def finding_entities(sentences):

    regexp_rules = [

        ## REGEX TABAC
        RegexpMatcherRule(regexp=r"\btaba(c|gisme)\b", label="statut_tabagisme"),
        RegexpMatcherRule(regexp=r"\btabagique\b", label="statut_tabagisme"),
        RegexpMatcherRule(regexp=r"\bcigare(tte)?[s]?\b", label="statut_tabagisme"),
        RegexpMatcherRule(regexp=r"\bfumeur[s]?\b", label="statut_tabagisme"),
        RegexpMatcherRule(regexp=r"\bfumeuse[s]?\b", label="statut_tabagisme"),
        RegexpMatcherRule(regexp=r"\bfume(e)?[s]?\b", label="statut_tabagisme"),
        RegexpMatcherRule(regexp=r"\bfume[r]?\b", label="statut_tabagisme"),

        ## REGEX ALCOOL
        RegexpMatcherRule(regexp=r"\balcool\b", label="statut_alcool"),
        RegexpMatcherRule(regexp=r"\bboit\b", label="statut_alcool"),
        RegexpMatcherRule(regexp=r"\balcoolique\b", label="statut_alcool"),
        RegexpMatcherRule(regexp=r"\bdependance\s*alcool\b", label="statut_alcool"),
        RegexpMatcherRule(regexp=r"\balcoolisme\b", label="statut_alcool"),

        ## REGEX STATUT MARITAL
        RegexpMatcherRule(regexp=r"\bmarie[e]?\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\bcelibataire\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\bdivorce[e]?\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\bveuf\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\bveuve\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\bpacse[e][s]?\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\bconcubinage\b", label="statut_marital"),
        RegexpMatcherRule(regexp=r"\b(vit|habite)\sseul(e)?\b", label="statut_marital"),

        ## REGEX ANTECEDENT
        #RegexpMatcherRule(regexp=r"\bantecedent(s)?\b", label="antecedent"),
]

    regexp_matcher = RegexpMatcher(rules=regexp_rules)
    entities = regexp_matcher.run(sentences)

    return entities,regexp_rules

In [318]:
"""
La fonction detecting_negation crée un détecteur de négation qui utilise des règles basées sur
des expressions régulières pour repérer les indicateurs de négation dans le texte. Les règles
couvrent les domaines du statut marital, du tabagisme, de l'alcool et des antécédents.
Le détecteur de négation est renvoyé en sortie de la fonction.
"""
def detecting_negation():
    neg_rules = [
        
        ## STATUT MARITAL
        NegationDetectorRule(regexp=r"\bn'est pas\b"),
        NegationDetectorRule(regexp=r"\bne vit pas\b"),
        NegationDetectorRule(regexp=r"\bn'habite pas\b"),
        NegationDetectorRule(regexp=r"\bni\s*marie(e)?\b"),
        NegationDetectorRule(regexp=r"\bni\s*divorce(e)?\b"),
        NegationDetectorRule(regexp=r"\bni\s*celibataire(e)?\b"),
        NegationDetectorRule(regexp=r"\bni\s*veu(f|ve)\b"),
        
        ## STATUT TABAGISME
        NegationDetectorRule(regexp=r"\bne\s*(semble|consomme|prend)\s*pas"),
        NegationDetectorRule(regexp=r"jamais"),
        NegationDetectorRule(regexp=r"arret"),
        NegationDetectorRule(regexp=r"ancien"),
        NegationDetectorRule(regexp=r"\b:\s*non\b"),
        NegationDetectorRule(regexp=r"ne\s*fume\s*pas"),
        NegationDetectorRule(regexp=r"\b0\s*(paquet[s]?|cigarette[s]?|jr[s]?|jour[s]?|boite[s]?)\b"),
        NegationDetectorRule(regexp=r"\b(non(-|\s)?fumeur),
        NegationDetectorRule(regexp=r"\b(non(-|\s)?fumeuse"),
        NegationDetectorRule(regexp=r"\bjamais(/s)*fum)"),
        NegationDetectorRule(regexp=r"\b(non(-|\s)?tabagique)"),
        NegationDetectorRule(regexp=r"\bni\s*tabac\b"),
        
        ## STATUT ALCOOL
        NegationDetectorRule(regexp=r"ne\s*boit\s*pas"),
        NegationDetectorRule(regexp=r"\bpas\s*alcool(ique)?\b"),
        NegationDetectorRule(regexp=r"\bpas\s*d'alcool\b"),


    ]
    
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector


In [319]:
"""
La fonction statut_extraction extrait les informations de statut de tabagisme, 
statut marital et statut d'alcool à partir d'un dictionnaire dico en parcourant les 
annotations, en calculant les proportions et en choisissant les statuts appropriés. 
Elle renvoie les statuts extraits.
"""

def statut_extraction(dico):

    ## Initialisation
    statut_tabagisme = "UNKNOWN"
    statut_marital = "UNKNOWN"
    statut_alcool = "UNKNOWN"
    # Nombre d'entité trouvé
    n_oui_tabac = 0
    n_non_tabac = 0
    n_oui_alcool = 0
    n_non_alcool = 0    
    # Proportion calculée à partir du nombre d'entité trouvé
    p_neg_tabac = 0
    p_pos_tabac = 0
    p_neg_alcool = 0
    p_pos_alcool = 0

    # On parcourt le dico pour analyser chaque annotation trouvé
    for ann in dico['anns']:
        # On récupère les valeurs is_negated et other_detected
        value_is_negated = ann['attrs'][0]['value']
        value_other_detected = ann['attrs'][1]['value']
        
        # Si l'entitée trouvée ne concerne pas le patient donc other_detected == True, 
        # on passe directement à l'annotation suivante
        if value_other_detected:
            continue
        
        # STATUT TABAGISME
        if ann['label']== "statut_tabagisme":
            #print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")
            # Si c'est une négation, on incrémente n_non_tabac de 1
            if value_is_negated == True:
                n_non_tabac += 1
            # Si ce n'est pas une négation, on incrémente n_oui_tabac de 1
            else:
                n_oui_tabac += 1

        # STATUT MARITAL   
        if ann['label']== "statut_marital":

            #print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")

            if ann['attrs'][0]['value'] == False:
                statut_marital = ann['text'].upper()
            else:
                statut_marital = "PAS"+" "+ann['text'].upper()
        
        # STATUT ALCOOL 
        if ann['label']== "statut_alcool":
            #print(f"Texte : {ann['text']}, Is_negated : {value_is_negated}, other_detected : {value_other_detected}")

            if value_is_negated == True:
                n_non_alcool += 1
            else:
                n_oui_alcool += 1

        

    if len(dico['anns']) != 0:        
        ### Porportion TABAC  
        p_neg_tabac = n_non_tabac/len(dico['anns'])
        p_pos_tabac = n_oui_tabac/len(dico['anns'])
        ### Porportion ALCOOL 
        p_neg_alcool = n_non_alcool/len(dico['anns'])
        p_pos_alcool = n_oui_alcool/len(dico['anns'])

           
    ## On choisit un statut en fonction des proportions calculées 
    if p_pos_tabac > p_neg_tabac:
        statut_tabagisme = "FUMEUR"

    if p_pos_tabac < p_neg_tabac:
        statut_tabagisme = "NON-FUMEUR"

    if p_pos_alcool > p_neg_alcool:
        statut_alcool = "ALCOOLIQUE"
        
    if p_pos_alcool < p_neg_alcool:
        statut_alcool = "NON-ALCOOLIQUE"

    return statut_tabagisme, statut_marital, statut_alcool

In [320]:
"""
La fonction clinical_case_recovery récupère tous les fichiers texte présents 
dans le dossier output_folder, les trie par ordre alphabétique, puis extrait 
et retourne les textes (= cas clinique) de chaque fichier sous forme d'une liste de cas cliniques.


def clinical_case_recovery(output_folder):
    # On récupère tous les fichiers texte dans le dossier
    txt_files = [f for f in os.listdir(output_folder) if f.endswith('.txt')]

    # On trie les fichiers par ordre alphabétique.
    txt_files_sorted = sorted(txt_files)

    textes = [] # liste de tous les cas cliniques

    # On ouvre et on extrait les textes dans textes
    for i in range(len(txt_files_sorted)):
        file_path = os.path.join(output_folder, txt_files_sorted[i])
        with open(file_path, 'r') as f:
            text = f.read()
        textes.append(text)
    return textes
"""

In [338]:
## Meme fonction mais renvoie un dico[nom fichier]= cas clinique

def clinical_case_recovery(output_folder):
    # On récupère tous les fichiers texte dans le dossier
    txt_files = [f for f in os.listdir(output_folder) if f.endswith('.txt')]

    # On trie les fichiers par ordre alphabétique.
    txt_files_sorted = sorted(txt_files)

    textes = {}  # dictionnaire de tous les cas cliniques

    # On ouvre et on extrait les textes dans textes
    for i in range(700):
        file_path = os.path.join(output_folder, txt_files_sorted[i])
        with open(file_path, 'r') as f:
            text = f.read()
        textes[txt_files_sorted[i]] = text
    return textes


# On exécute les fonctions sur 50 cas clinqiues

In [339]:
# TEST ON REAL CLINICAL CASES
output_folder = "clinical_case1"
textes = []
textes = clinical_case_recovery(output_folder)
# Listes pour stocker les données
data = []


# On parcourt clé (nom fichier), valeur (cas clinique) du dico textes
for fichier, cinical_case in textes.items():

    # PREPROCESSING THE TEXT (CLINICAL CASE)
    cinical_case = preprocess_text(cinical_case)

    # LOADING A TEXT DOCUMENT 
    doc = TextDocument(text=cinical_case)

    # SPLITTING A DOCUMENT IN SENTENCES
    sentences = splitting_doc_sentences(doc)

    # FINDING ENTITIES
    #On récupère les entities mais aussi les règles qui nous serviront à créer les entités
    entities,regexp_rules = finding_entities(sentences)

    # DETECTING NEGATION
    neg_detector = detecting_negation()

    # DETECTING TRUE OR WRONG NEGATION
    syntagmas = splitting_doc_syntagmas(sentences)
    # On recherche des negations dans les plus petites phrases (syntagmas) pour etre plus précis
    neg_detector.run(syntagmas)

    # DETECTING OTHER PATIENT OR NOT
    # Instanciation de la classe FamilyDetector avec une étiquette de sortie spécifique
    family_detector = FamilyDetector(output_label='other_detected')
    # On applique family_detector aux syntagmas
    family_detector.run(syntagmas)

    # CREATION OF ENTITIES
    regexp_matcher = RegexpMatcher(rules=regexp_rules, attrs_to_copy=["is_negated","other_detected"])
    entities = regexp_matcher.run(syntagmas)

    # AUGMENTING A DOCUMENT
    for entity in entities:
        doc.anns.add((entity))

    dico = doc.to_dict()

    # STATUT EXTRACTION
    statut_tabagisme,statut_marital,statut_alcool = statut_extraction(dico)

    # Remplissage de data
    
    if statut_marital != "UNKNOWN" or statut_tabagisme == "FUMEUR" or statut_alcool == "ALCOOLIQUE":
        
        # On ajoute les données des statuts à la liste
        data.append([fichier, cinical_case,statut_tabagisme, statut_alcool, statut_marital])

# On crée le DataFrame avec les colonnes nommées
df = pd.DataFrame(data, columns=["nom fichier", "cas clinique","statut tabagisme", "statut alcool", "statut marital"])


In [340]:
df.shape

(51, 5)

In [342]:
# On sauvegarde le DataFrame en fichier CSV pour annoter manuellement
df.head(50).to_csv("Extraction_statut_50_cas_cliniques_.csv", index=False)

In [343]:
import pandas as pd

# On modifie les options d'affichage de pandas 
pd.set_option('display.max_colwidth', None)

In [344]:
df

nom fichier   
0   EN101060_1.txt  \
1   EN101060_2.txt   
2   EN101060_3.txt   
3     EN101061.txt   
4     EN101074.txt   
5   EN101086_1.txt   
6   EN101096_0.txt   
7     EN101104.txt   
8     EN101122.txt   
9     EN101146.txt   
10    EN101160.txt   
11    EN101233.txt   
12    EN101241.txt   
13    EN101262.txt   
14    EN101303.txt   
15    EN101386.txt   
16  EN101409_1.txt   
17  EN101427_1.txt   
18    EN101507.txt   
19    EN101559.txt   
20    EN101612.txt   
21    EN101624.txt   
22    EN101665.txt   
23    EN101672.txt   
24  EN101693_1.txt   
25    EN101700.txt   
26    EN101728.txt   
27    EN101764.txt   
28    FR100051.txt   
29    FR100074.txt   
30    FR100092.txt   
31  FR100138_0.txt   
32  FR100138_1.txt   
33    FR100417.txt   
34    FR100423.txt   
35    FR100460.txt   
36    FR100489.txt   
37  FR100606_0.txt   
38    FR100614.txt   
39    FR100620.txt   
40    FR100629.txt   
41    FR100658.txt   
42    FR100660.txt   
43    FR100673.txt   
44    FR100689.txt   
45    FR100697.txt   
46    FR100708.txt   
47    FR100749.txt   
48    FR100817.txt   
49    FR100863.txt   
50    FR101011.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [346]:
df = pd.read_csv("Extraction_statut_50_cas_cliniques.csv")


In [351]:
# Les erreurs
df_filtre = df[df["statut tabagisme_true"].notna() | df["statut alcool_true"].notna() | df["statut marital_true"].notna()]
df_filtre

nom fichier   
7     EN101104.txt  \
11    EN101233.txt   
15    EN101386.txt   
24  EN101693_1.txt   
29    FR100074.txt   
43    FR100673.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [350]:
# Nombre d'erreur
df_filtre.shape

(6, 9)